# Notebook for meta-learning human motion from few random samples from discrete time series

# Meta-learning for regression tasks
1) 1-D regression: Sinewaves: predicting the sinewave that produces the 5 sampled points. learn a function $f_{\theta}(x) = y$ <br>
![title](sinewavesreg.png)
2)2-D Image completion: Given only 10-100 pixels of an image, regress the whole image. given a pixel location (x,y) regress the RGB vector values, $f_{\theta}(x, y) = [y_r, y_b, y_g]^T $
![title](imagesreg.png)
3)3-D Human behaviour understanding potentially learning the complete 3D mesh sequence motion from a few samples. Let M be a sequence of parametrized meshes by shape and pose coeffs [$m(\beta, \theta)_1, m(\beta, \theta)_2, ..., m(\beta, \theta)_T$]. and X = [$x_1, x_2...,x_T$] the discrete time series. The purpose is to learn the sequence M from N<<T samples only, that means learning how the pose parameters vary temporally.
\begin{align}
    Z &= randsample(M, N)\\
    f_{\theta}(Z) &= [ m(\beta, \theta_1), m(\beta, \theta_2),...m(\beta, \theta_T)]
\end{align}

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook
%matplotlib inline

import sys, os
import torch
import torch.nn as nn
import numpy as np
import glob
import learn2learn as l2l
from collections import OrderedDict
from human_body_prior.tools.omni_tools import copy2cpu as c2c
import warnings
from torch import optim
warnings.filterwarnings("ignore")

import trimesh
from human_body_prior.tools.omni_tools import colors
from human_body_prior.mesh import MeshViewer
from human_body_prior.mesh.sphere import points_to_spheres
from notebook_tools import show_image

In [2]:
comp_device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(comp_device)

cuda


In [3]:
def loss_meshes(target, predicted_poses, loss_fn, shots, bm):
    loss = torch.zeros(1).to(comp_device)
    betas = target['betas']
    dmpls = target['dmpls']
    poses = target['poses']
    for i in range(shots):
        #print("i: ", i)
        #print(predicted_poses[i].unsqueeze(0).shape)
        #print(betas.shape)
        #print(dmpls[i].shape)
        target = bm(pose_body=poses[i,3:66].unsqueeze(0), betas=betas, dmpls=dmpls[i].unsqueeze(0))
        mesh = bm(pose_body = predicted_poses[i].unsqueeze(0), betas=betas, dmpls=dmpls[i].unsqueeze(0))
        loss = loss + torch.abs(target.v - mesh.v).mean()
    loss /=shots
    return loss

def adaptation(x, y, learner, adaptation_steps, shots, loss_fn, bm):
    #separate data into adaptation/evaluation sets
    adaptation_indices = torch.zeros(x.size(0)).byte()
    adaptation_indices[torch.arange(shots)*2] = 1
    adaptation_data, evaluation_data = x[adaptation_indices], x[1 - adaptation_indices]
    
    adaptation_labels = {}
    evaluation_labels = {}
    adaptation_labels['poses'], evaluation_labels['poses'] = y['poses'][adaptation_indices], y['poses'][1 - adaptation_indices]
    adaptation_labels['trans'], evaluation_labels['trans'] = y['trans'][adaptation_indices], y['trans'][1 - adaptation_indices]
    adaptation_labels['dmpls'], evaluation_labels['dmpls'] = y['dmpls'][adaptation_indices], y['dmpls'][1 - adaptation_indices]
    adaptation_labels['betas'], evaluation_labels['betas'] = y['betas'], y['betas']
    
    for step in range(adaptation_steps):
        train_error = loss_meshes(adaptation_labels, learner(adaptation_data), loss_fn, shots, bm)
        with torch.no_grad():
            grads=torch.autograd.grad(train_error,learner.parameters())
            for p, t in zip(learner.parameters(), grads):
                p.data = p - alpha*t.detach().clamp_(-10, 10)
        
    predicted_poses = learner(evaluation_data)
    valid_error = loss_meshes(evaluation_labels, predicted_poses, loss_fn, shots, bm)
    return valid_error

# Load data, Body model used SMPLH (body, hands and dmpls)

In [4]:
from human_body_prior.body_model.body_model import BodyModel

bm_path = '/home/michalislazarou/PhD/amass/smplh/male/model.npz'
dmpl_path = '/home/michalislazarou/PhD/amass/dmpl/male/model.npz'
npz_bdata_path ='/home/michalislazarou/PhD/amass/datasets/EKUT/234/'
num_betas = 10 # number of body parameters
num_dmpls = 8 # number of DMPL parameters

bm = BodyModel(bm_path=bm_path, num_betas=num_betas, num_dmpls=num_dmpls, path_dmpl=dmpl_path).to(comp_device)
print("Body params: ", bm.pose_body.shape, "hand params: ", bm.pose_hand.shape, "dmpl: ", bm.dmpls.shape)
faces = c2c(bm.f)

Body params:  torch.Size([1, 63]) hand params:  torch.Size([1, 90]) dmpl:  torch.Size([1, 8])


# Create a Class to smaple Human motion examples from a dataset, in our case for this notebook we use the dataset EKUT(KIT Whole-Body Human Motion Database)

In [5]:
class HumanMotionDataset():
    def __init__(self, path_to_data, shots):
        self.img_path = path_to_data
        train_meshes, test_meshes = self.get_labels()
        self.train_meshes = train_meshes
        self.test_meshes = test_meshes
        self.shots = shots
        self.mesh = None
    
    def sample_task(self, train = 'train'):
        """ Sampling a task means sampling an mesh. """ 
        # choose image
        if train=='train':
            index = np.random.randint(len(self.train_meshes))
            self.mesh = self.train_meshes[index]
        else:
            index = np.random.randint(len(self.test_meshes))
            self.mesh = self.test_meshes[index]
        #return mesh
    
    def get_samples(self):
        x = np.sort(np.random.choice(self.mesh['poses'].shape[0], self.shots, replace=False))
        y = {}
        y['poses'] = self.mesh['poses'][x,:]
        y['trans'] = self.mesh['trans'][x,:]
        y['dmpls'] = self.mesh['dmpls'][x,:]
        y['betas'] = self.mesh['betas']
        return x, y
    def get_labels(self):
        data_list = []
        for data_file in glob.glob(os.path.join(self.img_path, '*.npz')):
            bdata = dict(np.load(data_file))
            bdata['poses'] = bdata['poses'][:200,:]
            bdata['trans'] = bdata['trans'][:200,:]
            bdata['dmpls'] = bdata['dmpls'][:200,:]
            data_list.append(bdata)
        return data_list[:70], data_list[70:] 

# In a similar way to the regression problem in sinewaves and image completion in: 
MAML:https://arxiv.org/pdf/1703.03400.pdf <br>
CAVIA:https://arxiv.org/pdf/1810.03642.pdf<br>
CNP: https://arxiv.org/pdf/1807.01613.pdf<br>
a regressor network is used as our meta-learning with the aim to regress the human motion from only a few samples of the complete motion. This can also be viewed as interpolation of motion.

In [6]:
class RegressorNet(nn.Module):
    def __init__(self):
        super(RegressorNet,self).__init__()
        self.net=nn.Sequential(OrderedDict([
            ('l1',nn.Linear(1,128)),
            ('relu1',nn.ReLU()),
            ('l2',nn.Linear(128,128)),
            ('relu2',nn.ReLU()),
            ('l3',nn.Linear(128,128)),
            ('relu3',nn.ReLU()),
            ('l4',nn.Linear(128,128)),
            ('relu4',nn.ReLU()),
            ('l5',nn.Linear(128,63)),
            #('relu5',nn.ReLU()),
            #('l6',nn.Linear(40,1))
        ]))
    
    def forward(self,x):
        return self.net(x)

# Load human motion dataset for single mesh example 

In [7]:
# import meta-dataset
data_list = []
for data_file in glob.glob(os.path.join(npz_bdata_path, '*.npz')):
    bdata = dict(np.load(data_file))
    bdata['poses'] = bdata['poses'][:200,:]
    bdata['trans'] = bdata['trans'][:200,:]
    bdata['dmpls'] = bdata['dmpls'][:200,:]
    data_list.append(bdata)
train_list, test_list = data_list[:70], data_list[70:] 
print(bdata['betas'].shape)
print(len(data_list), len(train_list), len(test_list))

(16,)
79 70 9


# Meta-learning hyperparameters initialized 

In [8]:
alpha=0.01
beta=0.001
shots=10
adaptation_steps=5
meta_batch_size = 10
num_iterations = 20000

# Single mesh regression:
Target mesh shown<br>
Note that only the pose parameters of the body are regressed, no pose parameters for hands or shape parameters

In [9]:
bdata = data_list[25]

fId = 147 # frame id of the mocap sequence

root_orient = torch.Tensor(bdata['poses'][fId:fId+1, :3]).to(comp_device) # controls the global root orientation
pose_body = torch.Tensor(bdata['poses'][fId:fId+1, 3:66]).to(comp_device) # controls the body
pose_hand = torch.Tensor(bdata['poses'][fId:fId+1, 66:]).to(comp_device) # controls the finger articulation
betas = torch.Tensor(bdata['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
dmpls = torch.Tensor(bdata['dmpls'][fId:fId+1]).to(comp_device) # controls soft tissue dynamics

body = bm(pose_body=pose_body, betas=betas, dmpls=dmpls)
target = trimesh.Trimesh(vertices=c2c(body.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
target.show()
#print(pose_body.shape)
#print(betas.shape)
#print(dmpls.shape)

# Initial starting mesh (depends on the weights of the regressor net). It can be easily initialized to T pose or another mesh.

In [11]:
#meta learnable parameters
regressor = RegressorNet()
regressor = regressor.to(comp_device)
for p in regressor.parameters():
    torch.nn.init.constant_(p.data, 0)
body_pose = torch.tensor(np.zeros((1, 63)).astype('float32'), requires_grad = True, device = comp_device)
#hand_pose = torch.tensor(np.zeros((1, 90)).astype('float32'), requires_grad = True, device = device)
#dmpls = torch.tensor(np.zeros((1, 8)).astype('float32'), requires_grad = True, device = device)
#maml = l2l.algorithms.MAML(body_pose, lr=alpha, first_order=False)
loss=nn.MSELoss()
opt = optim.Adam(regressor.parameters(), alpha)
mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
#body_mesh.show()
x=torch.tensor(fId, dtype=torch.float).unsqueeze(0).to(comp_device)
body_pose = regressor(x).unsqueeze(0)
mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
body_mesh.show()

# Learn the regressor parameters to predict the pose coefficients for the target mesh, iter=5000

In [11]:
for i in range(7000):
    
    body_pose = regressor(x).unsqueeze(0)
    
    #print(body_pose.shape, pose_body.shape)
    
    mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
    
    opt.zero_grad()
    
    loss = torch.abs(body.v - mesh.v).mean()
    
    loss.backward(retain_graph=True)
    
    opt.step()
    if i%200==0:
        print("iteration: ", i, "loss: ", loss.item())
   # mesh_view = smpl_model(beta.detach(), theta.detach())
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
print(body_mesh.center_mass)

iteration:  0 loss:  0.16636599600315094
iteration:  200 loss:  0.0035446167457848787
iteration:  400 loss:  0.002179093426093459
iteration:  600 loss:  0.0013270832132548094
iteration:  800 loss:  0.0010098946513608098
iteration:  1000 loss:  0.0012497295392677188
iteration:  1200 loss:  0.0017664076294749975
iteration:  1400 loss:  0.0010817509610205889
iteration:  1600 loss:  0.0016444290522485971
iteration:  1800 loss:  0.0016481020720675588
iteration:  2000 loss:  0.0007867119857110083
iteration:  2200 loss:  0.001258629490621388
iteration:  2400 loss:  0.001074694562703371
iteration:  2600 loss:  0.000898742291610688
iteration:  2800 loss:  0.0007484267698600888
iteration:  3000 loss:  0.0009433311061002314
iteration:  3200 loss:  0.0010593910701572895
iteration:  3400 loss:  0.0011194206308573484
iteration:  3600 loss:  0.0015953135443851352
iteration:  3800 loss:  0.0014361701905727386
iteration:  4000 loss:  0.0008017956861294806
iteration:  4200 loss:  0.0009691409068182111
i

# Regressor network mesh prediction

In [12]:
x1=torch.tensor(147, dtype=torch.float).unsqueeze(0).to(comp_device)
body_pose = regressor(x1).unsqueeze(0)
mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
body_mesh = trimesh.Trimesh(vertices=c2c(mesh.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
body_mesh.show()

# Target mesh shown

In [13]:
target.show()

# Regress 10 meshes as a time series signal with the same regressor network.
This can be thought of as learning a parametrized function H from time, t, to mesh, M.<br>

$$H_{\phi}(t_i) \rightarrow M_{t_i}$$

In [14]:
human_motion = HumanMotionDataset(npz_bdata_path, shots)
human_motion.sample_task()
x, y = human_motion.get_samples()
x=torch.tensor(x, dtype=torch.float).unsqueeze(1).to(comp_device)
#print(y['betas'].shape)
#convert to pytorch tensors
y['betas'] = torch.Tensor(y['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
y['trans'] = torch.Tensor(y['trans']).to(comp_device)
y['poses'] = torch.Tensor(y['poses']).to(comp_device)
y['dmpls'] = torch.Tensor(y['dmpls']).to(comp_device)
# use in learning loop
#target_pose = y['poses'][:,3:66]
#betas = y['betas']
#dmpls = y['dmpls']
print(y['betas'].shape)
print(y['poses'].shape)
print(y['dmpls'].shape)
print(x)

torch.Size([1, 10])
torch.Size([10, 156])
torch.Size([10, 8])
tensor([[ 13.],
        [ 43.],
        [ 44.],
        [ 77.],
        [ 97.],
        [107.],
        [138.],
        [148.],
        [154.],
        [164.]], device='cuda:0')


# View target mesh

In [15]:
frame=1
t = bm(pose_body=y['poses'][frame,3:66].unsqueeze(0), betas=y['betas'], dmpls=y['dmpls'][frame].unsqueeze(0))
target_mesh = trimesh.Trimesh(vertices=c2c(t.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
target_mesh.show()

In [16]:
for i in range(5000):
    
   # print(i)
    
    body_pose = regressor(x)
    
    #print(body_pose.shape)
    
    loss = loss_meshes(y, body_pose, nn.MSELoss(), shots, bm)
    
    #mesh = bm(pose_body=body_pose, betas=betas, dmpls=dmpls)
    
    opt.zero_grad()
    
    #loss = torch.abs(body.v - mesh.v).mean()
    
    loss.backward()
    
    opt.step()
    if i%1000==0:
        print("iteration: ", i, "loss: ", loss.item())
        #for j in range(body_pose.shape[0]):
         #   print("i: ",i," ", torch.sum(body_pose[j]).item())

iteration:  0 loss:  0.03291650861501694
iteration:  1000 loss:  0.022005917504429817
iteration:  2000 loss:  0.02200014889240265
iteration:  3000 loss:  0.021998904645442963
iteration:  4000 loss:  0.02204802818596363


# Regressor prediction 

In [17]:
frame = 1
reg = bm(pose_body=body_pose[frame].unsqueeze(0), betas=y['betas'], dmpls=y['dmpls'][frame].unsqueeze(0))
regressed_mesh = trimesh.Trimesh(vertices=c2c(reg.v[0]), faces=faces, vertex_colors=np.tile(colors['white'], (6890, 1)))
regressed_mesh.show()
#PROBLEM: meshes are the same for all prediction
#for i in range(body_pose.shape[0]):
 #   print("i: ",i," ", torch.sum(body_pose[i]).item())

# Meta-learning on a distribution of human motion data



In [9]:
regressor_meta = RegressorNet()
regressor_meta = regressor_meta.to(comp_device)
maml = l2l.algorithms.maml.MAML(regressor_meta, lr=alpha, first_order=False)
opt_meta = optim.Adam(maml.parameters(), beta)
loss_fn = nn.MSELoss()
human_distribution = HumanMotionDataset(npz_bdata_path, shots*2)

# Meta-training phase 

In [ ]:
# implementation of l2l framework for MAML, really slow

for iteration in range(50000):
    opt_meta.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy =0.0
    print("iteration: ", iteration)
    for task in range(meta_batch_size):
        print("task: ", task)
        learner = maml.clone()
        human_distribution.sample_task()
        x, y = human_distribution.get_samples()
        x=torch.tensor(x, dtype=torch.float).unsqueeze(1).to(comp_device)
        y['betas'] = torch.Tensor(y['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
        y['trans'] = torch.Tensor(y['trans']).to(comp_device)
        y['poses'] = torch.Tensor(y['poses']).to(comp_device)
        y['dmpls'] = torch.Tensor(y['dmpls']).to(comp_device)
        evaluation_error = adaptation(x, y, learner, adaptation_steps, shots, loss_fn, bm)
        
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
    for p in maml.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt.step()
    
    if iteration%5000==0:
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)

iteration:  0
task:  0
task:  1
task:  2
task:  3
task:  4
task:  5


In [ ]:
for iteration in range(50000):
    opt_meta.zero_grad()
    meta_train_error = 0.0
    meta_train_accuracy =0.0
    init_weights = [w.clone() for w in regressor_meta.parameters()]
    print("iteration: ", iteration)
    for task in range(meta_batch_size):
        #print(task)
        human_distribution.sample_task()
        x, y = human_distribution.get_samples()
        x=torch.tensor(x, dtype=torch.float).unsqueeze(1).to(comp_device)
        y['betas'] = torch.Tensor(y['betas'][:10][np.newaxis]).to(comp_device) # controls the body shape
        y['trans'] = torch.Tensor(y['trans']).to(comp_device)
        y['poses'] = torch.Tensor(y['poses']).to(comp_device)
        y['dmpls'] = torch.Tensor(y['dmpls']).to(comp_device)
        evaluation_error = adaptation(x, y, regressor_meta, adaptation_steps, shots, loss_fn, bm)
        
        evaluation_error.backward()
        meta_train_error += evaluation_error.item()
        
        for p, k in zip(regressor_meta.parameters(), init_weights):
            p.data = k
    for p in regressor_meta.parameters():
        p.grad.data.mul_(1.0 / meta_batch_size)
    opt_meta.step()
    
    if iteration%5000==0:
        print('Iteration', iteration)
        print('Meta Train Error', meta_train_error / meta_batch_size)

iteration:  0
Iteration 0
Meta Train Error 0.20759422779083253
iteration:  1
iteration:  2
iteration:  3
iteration:  4
iteration:  5
iteration:  6
iteration:  7
iteration:  8
iteration:  9
iteration:  10
iteration:  11
iteration:  12
iteration:  13
iteration:  14
iteration:  15
iteration:  16
iteration:  17
iteration:  18
iteration:  19
iteration:  20
iteration:  21
iteration:  22
iteration:  23
iteration:  24
iteration:  25
iteration:  26
iteration:  27
iteration:  28
iteration:  29
iteration:  30
iteration:  31
iteration:  32
iteration:  33
iteration:  34
iteration:  35
iteration:  36
iteration:  37
iteration:  38
iteration:  39
iteration:  40
iteration:  41
iteration:  42
iteration:  43
iteration:  44
iteration:  45
iteration:  46
iteration:  47
iteration:  48
iteration:  49
iteration:  50
iteration:  51
iteration:  52
iteration:  53
iteration:  54
iteration:  55
iteration:  56
iteration:  57
iteration:  58
iteration:  59
iteration:  60
iteration:  61
iteration:  62
iteration:  63
i

iteration:  516
iteration:  517
iteration:  518
iteration:  519
iteration:  520
iteration:  521
iteration:  522
iteration:  523
iteration:  524
iteration:  525
iteration:  526
iteration:  527
iteration:  528
iteration:  529
iteration:  530
iteration:  531
iteration:  532
iteration:  533
iteration:  534
iteration:  535
iteration:  536
iteration:  537
iteration:  538
iteration:  539
iteration:  540
iteration:  541
iteration:  542
iteration:  543
iteration:  544
iteration:  545
iteration:  546
iteration:  547
iteration:  548
iteration:  549
iteration:  550
iteration:  551
iteration:  552
iteration:  553
iteration:  554
iteration:  555
iteration:  556
iteration:  557
iteration:  558
iteration:  559
iteration:  560
iteration:  561
iteration:  562
iteration:  563
iteration:  564
iteration:  565
iteration:  566
iteration:  567
iteration:  568
iteration:  569
iteration:  570
iteration:  571
iteration:  572
iteration:  573
iteration:  574
iteration:  575
iteration:  576
iteration:  577
iteratio

iteration:  1027
iteration:  1028
iteration:  1029
iteration:  1030
iteration:  1031
iteration:  1032
iteration:  1033
iteration:  1034
iteration:  1035
iteration:  1036
iteration:  1037
iteration:  1038
iteration:  1039
iteration:  1040
iteration:  1041
iteration:  1042
iteration:  1043
iteration:  1044
iteration:  1045
iteration:  1046
iteration:  1047
iteration:  1048
iteration:  1049
iteration:  1050
iteration:  1051
iteration:  1052
iteration:  1053
iteration:  1054
iteration:  1055
iteration:  1056
iteration:  1057
iteration:  1058
iteration:  1059
iteration:  1060
iteration:  1061
iteration:  1062
iteration:  1063
iteration:  1064
iteration:  1065
iteration:  1066
iteration:  1067
iteration:  1068
iteration:  1069
iteration:  1070
iteration:  1071
iteration:  1072
iteration:  1073
iteration:  1074
iteration:  1075
iteration:  1076
iteration:  1077
iteration:  1078
iteration:  1079
iteration:  1080
iteration:  1081
iteration:  1082
iteration:  1083
iteration:  1084
iteration:  10

iteration:  1509
iteration:  1510
iteration:  1511
iteration:  1512
iteration:  1513
iteration:  1514
iteration:  1515
iteration:  1516
iteration:  1517
iteration:  1518
iteration:  1519
iteration:  1520
iteration:  1521
iteration:  1522
iteration:  1523
iteration:  1524
iteration:  1525
iteration:  1526
iteration:  1527
iteration:  1528
iteration:  1529
iteration:  1530
iteration:  1531
iteration:  1532
iteration:  1533
iteration:  1534
iteration:  1535
iteration:  1536
iteration:  1537
iteration:  1538
iteration:  1539
iteration:  1540
iteration:  1541
iteration:  1542
iteration:  1543
iteration:  1544
iteration:  1545
iteration:  1546
iteration:  1547
iteration:  1548
iteration:  1549
iteration:  1550
iteration:  1551
iteration:  1552
iteration:  1553
iteration:  1554
iteration:  1555
iteration:  1556
iteration:  1557
iteration:  1558
iteration:  1559
iteration:  1560
iteration:  1561
iteration:  1562
iteration:  1563
iteration:  1564
iteration:  1565
iteration:  1566
iteration:  15

iteration:  1991
iteration:  1992
iteration:  1993
iteration:  1994
iteration:  1995
iteration:  1996
iteration:  1997
iteration:  1998
iteration:  1999
iteration:  2000
iteration:  2001
iteration:  2002
iteration:  2003
iteration:  2004
iteration:  2005
iteration:  2006
iteration:  2007
iteration:  2008
iteration:  2009
iteration:  2010
iteration:  2011
iteration:  2012
iteration:  2013
iteration:  2014
iteration:  2015
iteration:  2016
iteration:  2017
iteration:  2018
iteration:  2019
iteration:  2020
iteration:  2021
iteration:  2022
iteration:  2023
iteration:  2024
iteration:  2025
iteration:  2026
iteration:  2027
iteration:  2028
iteration:  2029
iteration:  2030
iteration:  2031
iteration:  2032
iteration:  2033
iteration:  2034
iteration:  2035
iteration:  2036
iteration:  2037
iteration:  2038
iteration:  2039
iteration:  2040
iteration:  2041
iteration:  2042
iteration:  2043
iteration:  2044
iteration:  2045
iteration:  2046
iteration:  2047
iteration:  2048
iteration:  20

...

iteration:  27488
iteration:  27489
iteration:  27490
iteration:  27491
iteration:  27492
iteration:  27493
iteration:  27494
iteration:  27495
iteration:  27496
iteration:  27497
iteration:  27498
iteration:  27499
iteration:  27500
iteration:  27501
iteration:  27502
iteration:  27503
iteration:  27504
iteration:  27505
iteration:  27506
iteration:  27507
iteration:  27508
iteration:  27509
iteration:  27510
iteration:  27511
iteration:  27512
iteration:  27513
iteration:  27514
iteration:  27515
iteration:  27516
iteration:  27517
iteration:  27518
iteration:  27519
iteration:  27520
iteration:  27521
iteration:  27522
iteration:  27523
iteration:  27524
iteration:  27525
iteration:  27526
iteration:  27527
iteration:  27528
iteration:  27529
iteration:  27530
iteration:  27531
iteration:  27532
iteration:  27533
iteration:  27534
iteration:  27535
iteration:  27536
iteration:  27537
iteration:  27538
iteration:  27539
iteration:  27540
iteration:  27541
iteration:  27542
iteration:

iteration:  27944
iteration:  27945
iteration:  27946
iteration:  27947
iteration:  27948
iteration:  27949
iteration:  27950
iteration:  27951
iteration:  27952
iteration:  27953
iteration:  27954
iteration:  27955
iteration:  27956
iteration:  27957
iteration:  27958
iteration:  27959
iteration:  27960
iteration:  27961
iteration:  27962
iteration:  27963
iteration:  27964
iteration:  27965
iteration:  27966
iteration:  27967
iteration:  27968
iteration:  27969
iteration:  27970
iteration:  27971
iteration:  27972
iteration:  27973
iteration:  27974
iteration:  27975
iteration:  27976
iteration:  27977
iteration:  27978
iteration:  27979
iteration:  27980
iteration:  27981
iteration:  27982
iteration:  27983
iteration:  27984
iteration:  27985
iteration:  27986
iteration:  27987
iteration:  27988
iteration:  27989
iteration:  27990
iteration:  27991
iteration:  27992
iteration:  27993
iteration:  27994
iteration:  27995
iteration:  27996
iteration:  27997
iteration:  27998
iteration:

iteration:  28400
iteration:  28401
iteration:  28402
iteration:  28403
iteration:  28404
iteration:  28405
iteration:  28406
iteration:  28407
iteration:  28408
iteration:  28409
iteration:  28410
iteration:  28411
iteration:  28412
iteration:  28413
iteration:  28414
iteration:  28415
iteration:  28416
iteration:  28417
iteration:  28418
iteration:  28419
iteration:  28420
iteration:  28421
iteration:  28422
iteration:  28423
iteration:  28424
iteration:  28425
iteration:  28426
iteration:  28427
iteration:  28428
iteration:  28429
iteration:  28430
iteration:  28431
iteration:  28432
iteration:  28433
iteration:  28434
iteration:  28435
iteration:  28436
iteration:  28437
iteration:  28438
iteration:  28439
iteration:  28440
iteration:  28441
iteration:  28442
iteration:  28443
iteration:  28444
iteration:  28445
iteration:  28446
iteration:  28447
iteration:  28448
iteration:  28449
iteration:  28450
iteration:  28451
iteration:  28452
iteration:  28453
iteration:  28454
iteration:

iteration:  28856
iteration:  28857
iteration:  28858
iteration:  28859
iteration:  28860
iteration:  28861
iteration:  28862
iteration:  28863
iteration:  28864
iteration:  28865
iteration:  28866
iteration:  28867
iteration:  28868
iteration:  28869
iteration:  28870
iteration:  28871
iteration:  28872
iteration:  28873
iteration:  28874
iteration:  28875
iteration:  28876
iteration:  28877
iteration:  28878
iteration:  28879
iteration:  28880
iteration:  28881
iteration:  28882
iteration:  28883
iteration:  28884
iteration:  28885
iteration:  28886
iteration:  28887
iteration:  28888
iteration:  28889
iteration:  28890
iteration:  28891
iteration:  28892
iteration:  28893
iteration:  28894
iteration:  28895
iteration:  28896
iteration:  28897
iteration:  28898
iteration:  28899
iteration:  28900
iteration:  28901
iteration:  28902
iteration:  28903
iteration:  28904
iteration:  28905
iteration:  28906
iteration:  28907
iteration:  28908
iteration:  28909
iteration:  28910
iteration:

iteration:  29312
iteration:  29313
iteration:  29314
iteration:  29315
iteration:  29316
iteration:  29317
iteration:  29318
iteration:  29319
iteration:  29320
iteration:  29321
iteration:  29322
iteration:  29323
iteration:  29324
iteration:  29325
iteration:  29326
iteration:  29327
iteration:  29328
iteration:  29329
iteration:  29330
iteration:  29331
iteration:  29332
iteration:  29333
iteration:  29334
iteration:  29335
iteration:  29336
iteration:  29337
iteration:  29338
iteration:  29339
iteration:  29340
iteration:  29341
iteration:  29342
iteration:  29343
iteration:  29344
iteration:  29345
iteration:  29346
iteration:  29347
iteration:  29348
iteration:  29349
iteration:  29350
iteration:  29351
iteration:  29352
iteration:  29353
iteration:  29354
iteration:  29355
iteration:  29356
iteration:  29357
iteration:  29358
iteration:  29359
iteration:  29360
iteration:  29361
iteration:  29362
iteration:  29363
iteration:  29364
iteration:  29365
iteration:  29366
iteration: